In [1]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training, datasets
from chainer.training import extensions

import numpy as np

from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
X = iris.data
X = X.astype(np.float32)

In [3]:
Y = iris.target
Y = Y.flatten().astype(np.int32)

In [4]:
train ,test= datasets.split_dataset_random(chainer.datasets.TupleDataset(X,Y),100)
train_iter = chainer.iterators.SerialIterator(train, 10)
test_iter = chainer.iterators.SerialIterator(test, 1,repeat=False, shuffle=False)

In [5]:
class IrisModel(chainer.Chain):
    def __init__(self):
        super(IrisModel,self).__init__(
                l1 = L.Linear(4,100),
                l2 = L.Linear(100,100),
                l3 = L.Linear(100,3))

    def __call__(self,x):    
         h = F.relu(self.l1(x))
         h = F.relu(self.l2(h))
         return self.l3(h)

In [6]:
model = L.Classifier(IrisModel())
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

updater = training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (30, 'epoch'), out="result")
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport( ['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))
# trainer.extend(extensions.ProgressBar())

In [7]:
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           1.23951     1.03659               0.42           0.56                      
2           0.746208    0.694044              0.74           0.78                      
3           0.555498    0.61899               0.77           0.56                      
4           0.448438    0.501323              0.85           0.8                       
5           0.384808    0.43386               0.85           0.88                      
6           0.343886    0.397124              0.9            0.88                      
7           0.311689    0.357416              0.92           0.9                       
8           0.283019    0.319386              0.96           0.9                       
9           0.257954    0.290414              0.97           0.94                      
10          0.23681     0.268185              0.97           0.96                      
11          0.217634    0.245335  

In [8]:
F.softmax(np.array([test[0][0]])).data

array([[ 0.76944584,  0.18974303,  0.03136431,  0.00944675]], dtype=float32)

In [9]:
F.softmax(np.array([test[0][0]])).data.argmax(axis=1)

array([0])

In [10]:
y = model.predictor(np.array([[1,  1,  1,  1  ]]).astype(np.float32))
F.softmax(y).data

array([[ 0.25907731,  0.11319122,  0.6277315 ]], dtype=float32)

In [11]:
F.softmax(y).data.argmax(axis=1)

array([2])

In [12]:
F.softmax(model.predictor(np.array([[1,  1,  1,  1  ]]).astype(np.float32))).data

array([[ 0.25907731,  0.11319122,  0.6277315 ]], dtype=float32)